In [1]:
import itertools

import pandas as pd
from pysam import FastaFile
from Bio import Align
from Bio.Align import substitution_matrices
import orjson

In [2]:
source_fa = FastaFile('/Users/liang/Box/MyCPTAC/CPTAC_proteome_v2.0/DCC/RefSeq_20180629/RefSeq.20180629_Human_ucsc_hg38_cpdbnr_mito_264contams.fasta.gz')
target_fa = FastaFile('tracked_results/uniprot_reviewed_human_proteome_hgnc_only.v2020_03.fasta.gz')

In [3]:
aligner = Align.PairwiseAligner()
aligner.mode = "global"
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

In [4]:
print(aligner)

Pairwise sequence aligner with parameters
  substitution_matrix: <Array object at 0x7fe7984e1eb0>
  target_internal_open_gap_score: -10.000000
  target_internal_extend_gap_score: -0.500000
  target_left_open_gap_score: -10.000000
  target_left_extend_gap_score: -0.500000
  target_right_open_gap_score: -10.000000
  target_right_extend_gap_score: -0.500000
  query_internal_open_gap_score: -10.000000
  query_internal_extend_gap_score: -0.500000
  query_left_open_gap_score: -10.000000
  query_left_extend_gap_score: -0.500000
  query_right_open_gap_score: -10.000000
  query_right_extend_gap_score: -0.500000
  mode: global



In [5]:
def make_match_only_coords(alignment):
    # Extract the alignment to collect the matched coordinate segments.
    # This is done by tracing the alignment.path
    # using the code from Bio.Align.PairwiseAlignment.__format__
    # https://github.com/biopython/biopython/blob/14bf4f2aef448aff47b7b27cf1387eb4272f61db/Bio/Align/__init__.py#L1051
    aln_coords = []

    seq1 = alignment.target
    seq2 = alignment.query
    path = alignment.path
    end1, end2 = path[0]

    start1 = end1
    start2 = end2
    for end1, end2 in path[1:]:
        if end1 != start1 and end2 != start2:
            # Match or mismatch
            match_grps = itertools.groupby(
                c1 == c2 
                for c1, c2 in zip(seq1[start1:end1], seq2[start2:end2])
            )
            c1_start = start1
            c2_start = start2
            for m, cs in match_grps:
                l = len(list(cs))
                c1_end = c1_start + l
                c2_end = c2_start + l
                # Only take in the matching segments
                if m:
                    aln_coords.append((
                        (c1_start, c1_end),
                        (c2_start, c2_end)
                    ))
                c1_start = c1_end
                c2_start = c2_end
        start1 = end1
        start2 = end2
    return aln_coords

In [6]:
mapping_df = pd.read_table('tracked_results/mappings/refseq_20180629_to_uniprot_2020_03_mappable.tsv.gz')
mapping_df.head()

,refseq_prot_id,hgnc_id,source_uniparc_id,uniprot_acc,target_uniparc_id,mapping_approach
0,NP_000005.2,HGNC:7,UPI0000155718,P01023,UPI000014038F,global_seq_align
1,NP_000006.2,HGNC:7646,UPI000013DE51,P11245,UPI000011F28B,global_seq_align
2,NP_000007.1,HGNC:89,UPI00001251E6,P11310,UPI00001251E6,identical_sequence
3,NP_000008.1,HGNC:90,UPI000004A863,P16219,UPI000004A863,identical_sequence
4,NP_000009.1,HGNC:92,UPI00001251EF,P49748,UPI00001251EF,identical_sequence


In [7]:
source_seq = source_fa.fetch('NP_001007024.1').replace('U', 'X')
target_seq = target_fa.fetch('Q92813').replace('U', 'X')

In [18]:
alignment = aligner.align(source_seq, target_seq)[0]

In [19]:
aln_coords = make_match_only_coords(alignment)
aln_coords

[((0, 74), (0, 74)), ((110, 309), (74, 273))]

In [20]:
for source_slice, target_slice in aln_coords:
    assert source_seq[slice(*source_slice)] == target_seq[slice(*target_slice)]

In [22]:
coord_str = orjson.dumps(aln_coords).decode()
coord_str

'[[[0,74],[0,74]],[[110,309],[74,273]]]'

In [13]:
to_align_df = mapping_df[mapping_df['mapping_approach'] == 'global_seq_align']

In [14]:
def gen_align_coord_str(source_id, target_id):
    # Read it up about U (Selenocysteine)
    # https://en.wikipedia.org/wiki/Selenocysteine
    source_seq = source_fa.fetch(source_id).replace('U', 'X')
    target_seq = target_fa.fetch(target_id).replace('U', 'X')
    alignment = aligner.align(source_seq, target_seq)[0]
    aln_coords = make_match_only_coords(alignment)
    coord_str = orjson.dumps(aln_coords).decode()
    return coord_str

In [15]:
%%timeit
gen_align_coord_str('NP_009297.2', 'P00519')

10.7 ms ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%time
aln_results = []
for i, (source_id, target_id) in enumerate(zip(to_align_df['refseq_prot_id'], to_align_df['uniprot_acc']), 1):
    coord_str = gen_align_coord_str(source_id, target_id)
    aln_results.append(coord_str)
    if i % 1000 == 0:
        print(f"... processed {i:d} / {len(to_align_df)} mappings")

align_coord_series = pd.Series(
    aln_results,
    index=to_align_df.index,
    name='align_coord',
    dtype='string'
)
out_df = mapping_df.join(align_coord_series)

... processed 1000 / 23441 mappings
... processed 2000 / 23441 mappings
... processed 3000 / 23441 mappings
... processed 4000 / 23441 mappings
... processed 5000 / 23441 mappings
... processed 6000 / 23441 mappings
... processed 7000 / 23441 mappings
... processed 8000 / 23441 mappings
... processed 9000 / 23441 mappings
... processed 10000 / 23441 mappings
... processed 11000 / 23441 mappings
... processed 12000 / 23441 mappings
... processed 13000 / 23441 mappings
... processed 14000 / 23441 mappings
... processed 15000 / 23441 mappings
... processed 16000 / 23441 mappings
... processed 17000 / 23441 mappings
... processed 18000 / 23441 mappings
... processed 19000 / 23441 mappings
... processed 20000 / 23441 mappings
... processed 21000 / 23441 mappings
... processed 22000 / 23441 mappings
... processed 23000 / 23441 mappings
CPU times: user 3min 6s, sys: 5.83 s, total: 3min 11s
Wall time: 3min 11s


In [24]:
out_df.to_csv('tracked_results/mappings/refseq_20180629_to_uniprot_2020_03_mappable.coord_mapping.tsv.gz')